Import Libraries

In [1]:
from PyQt5.QtCore import *
import pandas as pd
from qgis.core import *
import os
import geopandas as gpd
import fiona
from shapely.geometry import Polygon

In [2]:
# Increase width of notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Read in Data

Data file paths may need to be changed based on Site

In [3]:
# Create directories within selected folder

# If tapping into Google Drive, folder may need to be availble offline

root = input("Enter file path to site folder in EPA Salt Marsh UAS Study with quotations -- ")
root = root[1:-1]
base = root + "\Jupyter_Working_Folder"
XY_Points_Folder = base + "\XY_Points"
Polygon_Folder = base + "\Polygons_1"
if not os.path.exists(Polygon_Folder): os.makedirs(Polygon_Folder)

Enter file path to site folder in EPA Salt Marsh UAS Study with quotations -- "G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\7. SaltMUAS_share\EPA Salt Marsh UAS Study\Essex Bay"


In [4]:
# Read in .cvs with Transects
# Keep quotations around file path in the input
read_data = input("Enter file path to ground truthing data .csv here -- ")
read_data = read_data[1:-1]

# skiprows argument may need to be tweaked depending on format of the .csv's headers
df = pd.read_csv(read_data)

Enter file path to ground truthing data .csv here -- "G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\7. SaltMUAS_share\EPA Salt Marsh UAS Study\Essex Bay\Ground Truthing\essex2june2021_revised_poly.csv"


In [5]:
# Observe number of rows to discard including RTC data
print("Preview of data:")
df.head()

Preview of data:


,PolygonNum,PointNum,Subclass,Northing,Easting,Altitude,Notes
0,13,27,21j,4723935.561,354911.571,0.778,poly
1,13,28,21j,4723939.532,354911.531,0.721,poly
2,13,29,21j,4723948.559,354911.791,0.705,poly
3,13,30,21j,4723956.996,354911.985,0.783,poly
4,13,31,21j,4723956.995,354911.993,0.789,poly


In [6]:
# Remove top rows of data if necessary
dat = input("If dataframe start at the correct point, enter 1, if not, enter 2 -- ")
if dat == "2":
    datos = df.iloc[1:]
    dato = pd.DataFrame(datos)
else: dato = pd.DataFrame(df)
dato.head()

If dataframe start at the correct point, enter 1, if not, enter 2 -- 1


,PolygonNum,PointNum,Subclass,Northing,Easting,Altitude,Notes
0,13,27,21j,4723935.561,354911.571,0.778,poly
1,13,28,21j,4723939.532,354911.531,0.721,poly
2,13,29,21j,4723948.559,354911.791,0.705,poly
3,13,30,21j,4723956.996,354911.985,0.783,poly
4,13,31,21j,4723956.995,354911.993,0.789,poly


In [7]:
# Rename column headers
dato.columns = ['Transect', 'Point Number', 'Subclass', 'Northing', 'Easting', 'Altitude', 'Notes']

In [8]:
filter_dato = dato
filter_dato

,Transect,Point Number,Subclass,Northing,Easting,Altitude,Notes
0,13,27,21j,4723935.561,354911.571,0.778,poly
1,13,28,21j,4723939.532,354911.531,0.721,poly
2,13,29,21j,4723948.559,354911.791,0.705,poly
3,13,30,21j,4723956.996,354911.985,0.783,poly
4,13,31,21j,4723956.995,354911.993,0.789,poly
5,13,32,21j,4723965.659,354912.273,0.702,poly
6,13,33,21j,4723966.891,354912.457,0.744,poly
7,13,34,21j,4723972.682,354912.779,0.751,poly
8,13,35,21j,4723974.696,354912.906,0.873,poly
9,13,36,21j,4723977.345,354912.685,0.774,poly


In [9]:


# Add in empty columns for Class and Raw Subclass attributes

# Create empty fields for transect angles, Class, and mapping unit

filter_dato['Class'] = "N/A"
filter_dato["Raw Subclass"] = "N/A"
filter_dato['Pre Angle'] = "N/A"
filter_dato['Post Angle'] = "N/A"
filter_dato['mu'] = "N/A"
filter_dato['Class mu'] = "N/A"
filter_dato["Pre/Post"] = "N/A"

# Read in the data
filter_dato.head()

,Transect,Point Number,Subclass,Northing,Easting,Altitude,Notes,Class,Raw Subclass,Pre Angle,Post Angle,mu,Class mu,Pre/Post
0,13,27,21j,4723935.561,354911.571,0.778,poly,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,13,28,21j,4723939.532,354911.531,0.721,poly,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,13,29,21j,4723948.559,354911.791,0.705,poly,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,13,30,21j,4723956.996,354911.985,0.783,poly,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,13,31,21j,4723956.995,354911.993,0.789,poly,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [10]:
# Parse out Class from Subclass and append to new list

classe = []
y = 0
for i in filter_dato['Subclass']:
    if str(filter_dato['Subclass'].iloc[y])[:1] == "2":
        classe.append(2)
        y += 1
    elif str(filter_dato['Subclass'].iloc[y])[:1] == "3":
        classe.append(3)
        y += 1
    else:
        classe.append(1)
        y += 1

In [11]:
# Parse out Raw Subclass data with only letters - no attribute letters

subraw = []
y = 0
for i in filter_dato["Subclass"]:
    subraw.append(str(i)[:2])

In [12]:
filter_dato['Class'] = classe
filter_dato["Raw Subclass"] = subraw

In [13]:
# Filter for null values where Transect == NaN
# filter_dato = dato[dato['Transect'].isnull()]

In [14]:
filter_dato

,Transect,Point Number,Subclass,Northing,Easting,Altitude,Notes,Class,Raw Subclass,Pre Angle,Post Angle,mu,Class mu,Pre/Post
0,13,27,21j,4723935.561,354911.571,0.778,poly,2,21,N/A,N/A,N/A,N/A,N/A
1,13,28,21j,4723939.532,354911.531,0.721,poly,2,21,N/A,N/A,N/A,N/A,N/A
2,13,29,21j,4723948.559,354911.791,0.705,poly,2,21,N/A,N/A,N/A,N/A,N/A
3,13,30,21j,4723956.996,354911.985,0.783,poly,2,21,N/A,N/A,N/A,N/A,N/A
4,13,31,21j,4723956.995,354911.993,0.789,poly,2,21,N/A,N/A,N/A,N/A,N/A
5,13,32,21j,4723965.659,354912.273,0.702,poly,2,21,N/A,N/A,N/A,N/A,N/A
6,13,33,21j,4723966.891,354912.457,0.744,poly,2,21,N/A,N/A,N/A,N/A,N/A
7,13,34,21j,4723972.682,354912.779,0.751,poly,2,21,N/A,N/A,N/A,N/A,N/A
8,13,35,21j,4723974.696,354912.906,0.873,poly,2,21,N/A,N/A,N/A,N/A,N/A
9,13,36,21j,4723977.345,354912.685,0.774,poly,2,21,N/A,N/A,N/A,N/A,N/A


In [15]:
# filter_dato.to_csv(Polygon_Folder + "\\Westport_29_Sept_2018_Ditch.csv")

Begin Specifying Individual Polygon Features Here

In [28]:
# Find Rows Where Column Value is Equal to ...

body = filter_dato.loc[filter_dato['Transect'] == 13]
body

# df.loc[df['column_name'] == some_value]

,Transect,Point Number,Subclass,Northing,Easting,Altitude,Notes,Class,Raw Subclass,Pre Angle,Post Angle,mu,Class mu,Pre/Post
0,13,27,21j,4723935.561,354911.571,0.778,poly,2,21,N/A,N/A,N/A,N/A,N/A
1,13,28,21j,4723939.532,354911.531,0.721,poly,2,21,N/A,N/A,N/A,N/A,N/A
2,13,29,21j,4723948.559,354911.791,0.705,poly,2,21,N/A,N/A,N/A,N/A,N/A
3,13,30,21j,4723956.996,354911.985,0.783,poly,2,21,N/A,N/A,N/A,N/A,N/A
4,13,31,21j,4723956.995,354911.993,0.789,poly,2,21,N/A,N/A,N/A,N/A,N/A
5,13,32,21j,4723965.659,354912.273,0.702,poly,2,21,N/A,N/A,N/A,N/A,N/A
6,13,33,21j,4723966.891,354912.457,0.744,poly,2,21,N/A,N/A,N/A,N/A,N/A
7,13,34,21j,4723972.682,354912.779,0.751,poly,2,21,N/A,N/A,N/A,N/A,N/A
8,13,35,21j,4723974.696,354912.906,0.873,poly,2,21,N/A,N/A,N/A,N/A,N/A
9,13,36,21j,4723977.345,354912.685,0.774,poly,2,21,N/A,N/A,N/A,N/A,N/A


In [29]:
# forestFilter = filter_dato.iloc[0:23]
# channelFilter = filter_dato.iloc[23:-1]
# channelFilter

In [30]:
# waterFilter = filter_dato

In [32]:
# Input Final Output Folder Path
rootOut = input("Enter Path of Site in UAS Data Collection (and Year if Applicable) to Write Final Output --- ")
rootOut = rootOut[1:-1]
workingFinal = rootOut + "\\Poly_Working"

Enter Path of Site in UAS Data Collection (and Year if Applicable) to Write Final Output --- "G:\.shortcut-targets-by-id\0B6-MI-dco6FLWkZmTDZ4MFhRU1k\7. SaltMUAS_share\UAS Data Collection\Essex Bay\GIS_Data\2021"


Generate Polygons

In [20]:
# # Specify name and site to write in file path
# # outName = input("Enter site name and type of polygon -- ")
# outPoly = workingFinal + r"\\RR_2019_Water_Poly.shp"

In [21]:
# # Create Temporary Forest Polygon

# northList = waterFilter['Northing'].tolist()
# eastList = waterFilter['Easting'].tolist()

# polygon_geom = Polygon(zip(eastList, northList))
# crs = {'init': 'epsg:26919'}
# polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom]) 
# print(polygon.geometry)

# polygon.to_file(filename = Polygon_Folder + "\\waterGeom.shp", driver = 'ESRI Shapefile')

In [22]:
# polygon

In [23]:
# # Read in temporary polygon with geometries and assign attributes

# gdf = gpd.read_file(Polygon_Folder + '\\waterGeom.shp')
# waterNotes = waterFilter.iloc[0]['Notes']
# waterSub = "22jr/22gjr"
# waterClass = waterFilter.iloc[0]['Class']
# waterRawSub = "22"
# gdf['Transect'] = "N/A"
# gdf['PointNum'] = "N/A"
# gdf['SubClass'] = waterSub
# gdf['Northing'] = "N/A"
# gdf['Easting'] = "N/A"
# gdf['Altitude'] = "N/A"
# gdf['Notes'] = waterNotes
# gdf['Class'] = waterClass
# gdf['Raw Subclass'] = waterRawSub
# gdf['Pre Angle'] = "N/A"
# gdf['Post Angle'] = "N/A"
# gdf['mu'] = "N/A"
# gdf['Class mu'] = "N/A"
# gdf['Pre/Post'] = "N/A"
# del gdf['FID']
# gdf.to_file(outPoly)

In [24]:
# EDIT THIS LINE FOR NAME

In [35]:
# Specify name and site to write in file path
# outName = input("Enter site name and type of polygon -- ")
outPoly = workingFinal + r"\\Essex_2_Jun_2021_poly.shp"

In [36]:
# Create Channel Polygon

northList = body['Northing'].tolist()
eastList = body['Easting'].tolist()

polygon_geom = Polygon(zip(eastList, northList))
crs = {'init': 'epsg:26919'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom]) 
print(polygon.geometry)

polygon.to_file(filename = Polygon_Folder + "\\poly.shp", driver = 'ESRI Shapefile')

0    POLYGON ((354911.571 4723935.561, 354911.531 4...
Name: geometry, dtype: geometry


In [38]:
# Read in temporary polygon with geometries and assign attributes

gdf = gpd.read_file(Polygon_Folder + '\\poly.shp')
bodyNotes = body.iloc[0]['Notes']
bodySub = body.iloc[0]['Subclass']
bodyClass = body.iloc[0]['Class']
bodyRawSub = body.iloc[0]['Raw Subclass']
gdf['Transect'] = "N/A"
gdf['PointNum'] = "N/A"
gdf['SubClass'] = bodySub
gdf['Northing'] = "N/A"
gdf['Easting'] = "N/A"
gdf['Altitude'] = "N/A"
gdf['Notes'] = bodyNotes
gdf['Class'] = bodyClass
gdf['Raw Subclass'] = bodyRawSub
gdf['Pre Angle'] = "N/A"
gdf['Post Angle'] = "N/A"
gdf['mu'] = "N/A"
gdf['Class mu'] = "N/A"
gdf['Pre/Post'] = "N/A"
del gdf['FID']
gdf.to_file(outPoly)